In [1]:
import os
import sys
import math
import random
import timeit
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

%matplotlib notebook
sys.path.append("lib") # go to parent dir
import pickle

import subprocess
import nibabel
import PIL.Image as Image

import lib.motion as motion
import lib.head_motion
import lib.models as models

W1114 16:21:28.082671 140447965099776 deprecation_wrapper.py:119] From /data/vision/polina/scratch/nmsingh/dev/Motion-Experiments/lib/models.py:4: The name tf.keras.layers.CuDNNGRU is deprecated. Please use tf.compat.v1.keras.layers.CuDNNGRU instead.

W1114 16:21:28.083785 140447965099776 deprecation_wrapper.py:119] From /data/vision/polina/scratch/nmsingh/dev/Motion-Experiments/lib/models.py:4: The name tf.keras.layers.CuDNNLSTM is deprecated. Please use tf.compat.v1.keras.layers.CuDNNLSTM instead.

Using TensorFlow backend.


In [2]:
n = 64
num_lines = None
batch_size = 8

# Set up model
model = models.get_theta_k_model((n,n,2),single=True)
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.00002,rho=0.9),
        loss='mean_squared_error',
        metrics=[keras.metrics.mae])

# Load trained model weights

training_dir = 'training/ismrm/BRAIN-ALL-CONTIGUOUS-PATCHFalse-PARAMETERIZED_SINGLE_THETA-relu-FREQUENCY_INDOMAIN-SINGLE_THETA_K_OUTDOMAIN-False-5000epoch-64-*single_theta_k_256/'
model.load_weights(training_dir+'cp-0460.ckpt')

W1114 16:21:29.175992 140447965099776 deprecation.py:506] From /data/vision/polina/scratch/nmsingh/anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
img_dir = '/data/vision/polina/scratch/nmsingh/ADNI-data/uncropped/adni-split-by-subj-slice128-axial/train/origs/'
imgs = os.listdir(img_dir)
i = 0
img = np.load(img_dir+imgs[i])
img = np.rot90(img['vol_data'])
n = 64
sl_data = np.array(Image.fromarray(img).resize((n,n)))
sl_data = sl_data - sl_data.mean()
sl_data = sl_data/np.max(sl_data)

n = 64
k_line = np.random.randint(0,n)
k_vect = np.zeros(n)

k_line = 38#np.random.randint(0,n)
k_vect[k_line] = 1

num_pix = np.zeros((n,2))
num_pix[k_line:,0] = np.random.random()*20-10
num_pix[k_line:,1] = np.random.random()*20-10
angle = np.zeros(n)
angle[k_line:] = np.random.random()*90-45

corrupted_img,corrupted_k = motion.add_rotation_and_translation(sl_data,angle,num_pix,return_k=True)
corrupted_k = np.expand_dims(corrupted_k,-1)
corrupted_k_re = np.real(corrupted_k)
corrupted_k_im = np.imag(corrupted_k)
corrupted_k = np.concatenate([corrupted_k_re,corrupted_k_im], axis=2)

true_outputs = np.expand_dims(np.array([num_pix[-1,0],num_pix[-1,1],angle[-1],np.array(k_line)]),axis=0)
network_input = corrupted_k

n = 64
num_lines = None
batch_size = 8

# Set up model
model = models.get_theta_k_model((n,n,2),single=True)
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.00002,rho=0.9),
        loss='mean_squared_error',
        metrics=[keras.metrics.mae])

# Load trained model weights

training_dir = 'training/ismrm/BRAIN-ALL-CONTIGUOUS-PATCHFalse-PARAMETERIZED_SINGLE_THETA-relu-FREQUENCY_INDOMAIN-SINGLE_THETA_K_OUTDOMAIN-False-5000epoch-64-*single_theta_k_256/'
model.load_weights(training_dir+'cp-0460.ckpt')

model.predict(np.expand_dims(network_input,0))
batched_input = np.expand_dims(network_input,0)

In [4]:
cumtime = 0

a = time.time()
for i in range(100):
    a = time.time()
    model.predict(batched_input)
    cumtime+=time.time()-a

print(cumtime/100)

0.02177851438522339
